In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fandaoerji/cow-eartag-detection-dataset")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/cow-eartag-detection-dataset


In [ ]:
%cd /kaggle/input/cow-eartag-detection-dataset
!ls

/kaggle/input/cow-eartag-detection-dataset
'Cow eartag detection dataset'


In [ ]:
%cd Cow eartag detection dataset
!ls

/kaggle/input/cow-eartag-detection-dataset/Cow eartag detection dataset
Images	Labels


In [ ]:
%cd ultralytics

fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/yolov5/ultralytics


In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -r requirements.txt
!pip install ultralytics --upgrade
%cd ..

fatal: destination path 'ultralytics' already exists and is not an empty directory.
/root/.cache/kagglehub/datasets/fandaoerji/cow-eartag-detection-dataset/versions/1/Cow eartag detection dataset/ultralytics
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
/root/.cache/kagglehub/datasets/fandaoerji/cow-eartag-detection-dataset/versions/1/Cow eartag detection dataset


In [ ]:
%cd /root/.cache/kagglehub/datasets/fandaoerji/cow-eartag-detection-dataset/versions/1/Cow eartag detection dataset/Images

/root/.cache/kagglehub/datasets/fandaoerji/cow-eartag-detection-dataset/versions/1/Cow eartag detection dataset/Images


In [ ]:
!ls

In [ ]:
from sklearn.model_selection import train_test_split
from glob import glob
import os

images_path = '/kaggle/input/cow-eartag-detection-dataset/Cow eartag detection dataset/Images'
labels_path = '/kaggle/input/cow-eartag-detection-dataset/Cow eartag detection dataset/Labels'

contents = os.listdir(labels_path)

print(len(contents))

2451


In [ ]:
# Filter images that have a matching label
image_files = glob(os.path.join(images_path, '*.jpg'))
image_files = [img for img in image_files if os.path.exists(
    os.path.join(labels_path, os.path.basename(img).replace('.jpg', '.txt'))
)]

print(f"Found {len(image_files)} images with matching labels.")

# Split into train/val sets
train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=42)

Found 2451 images with matching labels.


In [ ]:
!ls

Images	Labels


In [ ]:
%cd ..

/kaggle/input/cow-eartag-detection-dataset


In [ ]:
for split in ['train', 'val']:
    os.makedirs(f'/content/cow_dataset/images/{split}', exist_ok=True)
    os.makedirs(f'/content/cow_dataset/labels/{split}', exist_ok=True)


In [ ]:
!ls

'Cow eartag detection dataset'


In [ ]:
# Function to move files
def move_files(image_list, split):
    for img_path in image_list:
        name = os.path.basename(img_path)
        label_name = name.replace('.jpg', '.txt')

        label_src = os.path.join(labels_path, label_name)
        img_dst = f'/content/cow_dataset/images/{split}/{name}'
        label_dst = f'/content/cow_dataset/labels/{split}/{label_name}'

        # Copy image and its corresponding label
        if os.path.exists(label_src):
            shutil.copy(img_path, img_dst)
            shutil.copy(label_src, label_dst)

# Move files to the dataset
move_files(train_images, 'train')
move_files(val_images, 'val')

print(f"Train set: {len(train_images)} images")
print(f"Val set: {len(val_images)} images")



Train set: 1960 images
Val set: 491 images


In [ ]:
# STEP 5: Create data.yaml for YOLOv9
with open('/content/cow.yaml', 'w') as f:
    f.write("""\
path: cow_dataset
train: /content/cow_dataset/images/train
val: /content/cow_dataset/images/val

nc: 1
names: ['tag']
""")

In [ ]:
%cd /content

/content


In [ ]:
data = os.listdir('/content/cow_dataset/labels/val')

print(len(data))

893


In [ ]:
!yolo task=detect mode=train model=yolov9t.pt data=/content/cow.yaml epochs=10 imgsz=640 batch=16

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov9t.pt, data=/content/cow.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, fo

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train3/weights/best.pt source=/content/barn3.png imgsz=640


Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv9t summary (fused): 197 layers, 1,970,979 parameters, 0 gradients, 7.6 GFLOPs

image 1/1 /content/barn3.png: 448x640 3 tags, 51.6ms
Speed: 4.1ms preprocess, 51.6ms inference, 307.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!zip -r /content/yolov9_train3_weights.zip /content/runs/detect/train3/weights


  adding: content/runs/detect/train3/weights/ (stored 0%)
  adding: content/runs/detect/train3/weights/last.pt (deflated 16%)
  adding: content/runs/detect/train3/weights/best.pt (deflated 16%)


In [ ]:
from google.colab import files
files.download('/content/yolov9_train3_weights.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>